In [1]:
import numpy as np
import pandas as pd
# from skimage import imread
from matplotlib.image import imread
import matplotlib.pyplot as plt
import os
import cv2
import random
import pickle
from sklearn.metrics import confusion_matrix, recall_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, callbacks
import sys
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.utils import shuffle

import utils as ut

In [2]:
df_train_pred = pd.read_csv('./data/train/train_set.csv')
df_test_pred = pd.read_csv('./data/test/test_set.csv')
ROOT_PATH = os.getcwd()
TRAIN_PATH = "./data/img/train/"
TEST_PATH = "./data/img/test/"
feel_names = ['angry','disgust','fear','happy','neutral','sad','surprise']
feel_names_dict = {feel_name:i for i, feel_name in enumerate(feel_names)}

In [3]:
### New: only in gray scale. Should be more efficient, again just for the first time
# x_train, y_train = ut.train_image_reader(TRAIN_PATH,'./data//processed/train/x1.pkl','./data//processed/train/y1.pkl')
x_test,df_test = ut.test_image_reader(TEST_PATH,'./data/processed/test/x1.pkl')

Opening image #0
Opening image #1
Opening image #2
Opening image #3
Opening image #4
Opening image #5
Opening image #6
Opening image #7
Opening image #8
Opening image #9
Opening image #10
Opening image #11
Opening image #12
Opening image #13
Opening image #14
Opening image #15
Opening image #16
Opening image #17
Opening image #18
Opening image #19
Opening image #20
Opening image #21
Opening image #22
Opening image #23
Opening image #24
Opening image #25
Opening image #26
Opening image #27
Opening image #28
Opening image #29
Opening image #30
Opening image #31
Opening image #32
Opening image #33
Opening image #34
Opening image #35
Opening image #36
Opening image #37
Opening image #38
Opening image #39
Opening image #40
Opening image #41
Opening image #42
Opening image #43
Opening image #44
Opening image #45
Opening image #46
Opening image #47
Opening image #48
Opening image #49
Opening image #50
Opening image #51
Opening image #52
Opening image #53
Opening image #54
Opening image #55
Op

In [11]:
df_test

,image,pixels,id_img
0,10004.jpg,"[[0.00392156862745098, 0.00392156862745098, 0....",10004
1,10019.jpg,"[[0.6235294117647059, 0.615686274509804, 0.619...",10019
2,10023.jpg,"[[0.7647058823529411, 0.7764705882352941, 0.75...",10023
3,10029.jpg,"[[0.1607843137254902, 0.18823529411764706, 0.2...",10029
4,1003.jpg,"[[0.17254901960784313, 0.12941176470588237, 0....",1003
...,...,...,...
7061,9977.jpg,"[[0.6980392156862745, 0.5450980392156862, 0.50...",9977
7062,9979.jpg,"[[0.9529411764705882, 0.9529411764705882, 0.93...",9979
7063,9982.jpg,"[[0.796078431372549, 0.8980392156862745, 0.917...",9982
7064,9984.jpg,"[[0.2235294117647059, 0.2196078431372549, 0.22...",9984


In [10]:
df_test['id_img'] = df_test['image'].str.replace('.jpg',"")

In [13]:
x_train = pickle.load(open('./data/processed/train/x1.pkl','rb'))
y_train = pickle.load(open('./data/processed/train/y1.pkl','rb'))
# x_test = pickle.load(open('./data/processed/test/x1.pkl','rb'))

In [31]:
x_train.shape

(28821, 48, 48)

## Data augmentation of disgust

In [32]:
datagen = ImageDataGenerator(
    rotation_range=20,  # Rotación aleatoria entre -20 y 20 grados
    width_shift_range=0.2,  # Desplazamiento horizontal aleatorio
    height_shift_range=0.2,  # Desplazamiento vertical aleatorio
    shear_range=0.2,  # Cortes aleatorios
    zoom_range=0.2,  # Zoom aleatorio
    horizontal_flip=True,  # Volteo horizontal
    fill_mode='nearest'  # Rellenar píxeles vacíos con el valor más cercano
)

In [33]:
X_disgust = x_train[y_train == 1]
desired_aug = 10
X_disgust_aug = []
y_disgust_aug = []

for img in X_disgust:
    img = img.reshape((1, img.shape[0], img.shape[1], 1))
    count = 0
    for batch in datagen.flow(img, batch_size=1):
        X_disgust_aug.append(batch[0])  # batch[0] contiene la imagen aumentada
        y_disgust_aug.append(1)
        count += 1
        if count >= desired_aug:  # Generar el número deseado de nuevas imágenes
            break
X_disgust_aug = np.array(X_disgust_aug)
y_disgust_aug = np.array(y_disgust_aug)
x_train = x_train.reshape(-1,48,48,1)
x_train = np.concatenate((x_train, X_disgust_aug), axis=0)
y_train = np.concatenate((y_train, y_disgust_aug), axis=0)
x_train, y_train = shuffle(x_train, y_train, random_state=42)
np.unique(y_train,return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6]),
 array([3993, 4796, 4103, 7164, 4982, 4938, 3205]))

In [34]:
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size=0.1,random_state=42)

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(48, (3, 3), activation='relu', input_shape=(48, 48,1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(96, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.6))
model.add(layers.Flatten())

# Capa densa con Dropout
model.add(layers.Dense(48, activation='relu'))
model.add(layers.Dropout(0.6))  # Dropout para reducir overfitting

model.add(layers.Dense(7, activation='softmax'))

# Compilar el modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



c:\Users\raulg\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [36]:
history = model.fit(x_train,y_train,epochs=50,batch_size = 64,validation_data=(x_val,y_val),
          callbacks=callbacks.EarlyStopping(patience=5)
          )

Epoch 1/50
467/467 ━━━━━━━━━━━━━━━━━━━━ 18s 36ms/step - accuracy: 0.2157 - loss: 1.9000 - val_accuracy: 0.3546 - val_loss: 1.7496
Epoch 2/50
467/467 ━━━━━━━━━━━━━━━━━━━━ 17s 36ms/step - accuracy: 0.3223 - loss: 1.7328 - val_accuracy: 0.4480 - val_loss: 1.5414
Epoch 3/50
467/467 ━━━━━━━━━━━━━━━━━━━━ 16s 34ms/step - accuracy: 0.3742 - loss: 1.6156 - val_accuracy: 0.4652 - val_loss: 1.4897
Epoch 4/50
467/467 ━━━━━━━━━━━━━━━━━━━━ 16s 34ms/step - accuracy: 0.3970 - loss: 1.5548 - val_accuracy: 0.4806 - val_loss: 1.4607
Epoch 5/50
467/467 ━━━━━━━━━━━━━━━━━━━━ 16s 35ms/step - accuracy: 0.4081 - loss: 1.5243 - val_accuracy: 0.4962 - val_loss: 1.3827
Epoch 6/50
467/467 ━━━━━━━━━━━━━━━━━━━━ 17s 36ms/step - accuracy: 0.4169 - loss: 1.4896 - val_accuracy: 0.5089 - val_loss: 1.3510
Epoch 7/50
467/467 ━━━━━━━━━━━━━━━━━━━━ 17s 36ms/step - accuracy: 0.4361 - loss: 1.4528 - val_accuracy: 0.5161 - val_loss: 1.3256
Epoch 8/50
467/467 ━━━━━━━━━━━━━━━━━━━━ 17s 36ms/step - accuracy: 0.4402 - loss: 1.4343 - 

In [15]:
# model.save("./models/model_3.keras")
model = models.load_model("./models/model_3.keras")

In [50]:
df_test_pred = df_test_pred.sort_values('id_img')
df_test_pred

,id_img
3805,8
5480,20
1449,21
388,23
3189,30
...,...
3542,35869
6911,35872
6050,35873
766,35881


In [ ]:
x_test = x_test.reshape(-1,48,48,1)
y_pred = model.predict(x_test).argmax(axis=1)
y_pred_names = [feel_names[x] for x in y_pred]
df_test['label'] = y_pred_names
df_test.head(5)


221/221 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step


,image,pixels,id_img,label
0,10004.jpg,"[[0.00392156862745098, 0.00392156862745098, 0....",10004,angry
1,10019.jpg,"[[0.6235294117647059, 0.615686274509804, 0.619...",10019,sad
2,10023.jpg,"[[0.7647058823529411, 0.7764705882352941, 0.75...",10023,sad
3,10029.jpg,"[[0.1607843137254902, 0.18823529411764706, 0.2...",10029,angry
4,1003.jpg,"[[0.17254901960784313, 0.12941176470588237, 0....",1003,sad


In [17]:
df_test[['id_img','label']].to_csv('./data/results/results_3.csv',index=False)

In [52]:
df_test_pred

,id_img,label
3805,8,angry
5480,20,sad
1449,21,sad
388,23,angry
3189,30,sad
...,...,...
3542,35869,happy
6911,35872,happy
6050,35873,angry
766,35881,neutral


In [42]:
x_train.shape

(29862, 48, 48, 1)

In [43]:
x_test.shape

(7066, 48, 48)